In [3]:
import odrive
from odrive.enums import *
import time
import numpy as np
import import_ipynb
from odrive_function import od, gait

## Connect to Odrive

In [ ]:
odrv2 = od.connect_odrive("335C315E3536")
odrv1 = od.connect_odrive("206A33A5304B")

m0  = odrv1.axis0
m1  = odrv1.axis1
m00 = odrv2.axis0
m11 = odrv2.axis1

## Setup

In [ ]:
od.SetParameter(odrv1)
od.SetParameter(odrv2)
print("Parameter Set!")

In [ ]:
print("odrive1")
od.CheckMotorError(odrv1)
print("odrive2")
od.CheckMotorError(odrv2)

od.MotorClearError(odrv1)
od.MotorClearError(odrv2)
print("Error Clear!")

## Motor Commands

In [ ]:
od.Calibration(odrv1)
od.Calibration(odrv2)

In [ ]:
od.ClosedLoopControlMode(odrv1)
od.ClosedLoopControlMode(odrv2)

In [ ]:
od.IdleMode(odrv1)
od.IdleMode(odrv2)

# Define zero angle (Joint space)

In [ ]:
m0_home_pos = od.SetMotorHomePosition(m0, m1)[0]
m1_home_pos = od.SetMotorHomePosition(m0, m1)[1]
print("Odrv1 \nR:", m0_home_pos, "\nL:", m1_home_pos)

In [ ]:
m00_home_pos = od.SetMotorHomePosition(m00, m11)[0]
m11_home_pos = od.SetMotorHomePosition(m00, m11)[1]
print("Odrv2 \nR:", m00_home_pos, "\nL:", m11_home_pos)

# Define tiptoe home location (Joint space)

In [ ]:
#odrv1
theta_R_eq_d = od.SetMotorEquilibriumPosition(m0, m1, m0_home_pos, m1_home_pos)[0]
theta_L_eq_d = od.SetMotorEquilibriumPosition(m0, m1, m0_home_pos, m1_home_pos)[1]
print("Odrv1 \nR:", theta_R_eq_d, "\nL:", theta_L_eq_d)

In [ ]:
#odrv2
theta_R_eq_d = od.SetMotorEquilibriumPosition(m00, m11, m00_home_pos, m11_home_pos)[0]
theta_L_eq_d = od.SetMotorEquilibriumPosition(m00, m11, m00_home_pos, m11_home_pos)[1]
print("Odrv2 \nR:", theta_R_eq_d, "\nL:", theta_L_eq_d)

## Space Convertor

In [ ]:
def from_joint_get_pos(m0_home_pos, m1_home_pos): #return current xy
    
    thetaR = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
    thetaL = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)

    x = FK_fast(thetaR, thetaL)[0]
    y = FK_fast(thetaR, thetaL)[1]
    
    return np.array([x, y])

def from_pos_get_joint(x, y, m0_home_pos, m1_home_pos): #xy ->encoder

    theta_R = IK_5_link(x, y)[1]
    theta_L = IK_5_link(x, y)[0]
    
    m0_pos = -(theta_R - np.pi)/(2*np.pi)*8192 + m0_home_pos
    m1_pos = -theta_L/(2*np.pi)*8192 + m1_home_pos
    
    return np.array([m0_pos, m1_pos])

## Test run (Positional Control)

In [ ]:
%matplotlib qt

In [ ]:
ControllerPositionControl(odrv1)
ControllerPositionControl(odrv2)

ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)

# Get ready to start---------------------------------------------------
phase_t = 1

m0_pos = from_pos_get_joint(RightStanceTrajectory(0, phase_t)[0], RightStanceTrajectory(0, phase_t)[1]-0.02, m0_home_pos, m1_home_pos)[0]
m1_pos = from_pos_get_joint(RightStanceTrajectory(0, phase_t)[0], RightStanceTrajectory(0, phase_t)[1]-0.02, m0_home_pos, m1_home_pos)[1]
m00_pos= from_pos_get_joint(LeftSwingTrajectory(0, phase_t)[0], LeftSwingTrajectory(0, phase_t)[1]-0.01, m00_home_pos, m11_home_pos)[0]
m11_pos= from_pos_get_joint(LeftSwingTrajectory(0, phase_t)[0], LeftSwingTrajectory(0, phase_t)[1]-0.01, m00_home_pos, m11_home_pos)[1]

m0.controller.set_pos_setpoint(m0_pos,0,0)
m1.controller.set_pos_setpoint(m1_pos,0,0)
m00.controller.set_pos_setpoint(m00_pos,0,0)
m11.controller.set_pos_setpoint(m11_pos,0,0)

In [ ]:
# Run test-------------------------------------------------------------
k = 1
sr = 0.1
phase_t = 1 
t0 = time.time()

ControllerCurrentControl(odrv1)
ControllerCurrentControl(odrv2)

#(test_cycle, swing_time, stance_time)
run_positional_step_test(8,phase_t,phase_t) 

print("time spent:{}sec".format(time.time() - t0))

In [ ]:
IdleMode(odrv1)
IdleMode(odrv2)

## Test Run (Virtual Compliance)

In [ ]:
ControllerPositionControl(odrv1)
ControllerPositionControl(odrv2)

ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)

# Get ready to start---------------------------------------------------
phase_t = 1

m0_pos = from_pos_get_joint(RightStanceTrajectory(0, phase_t)[0], RightStanceTrajectory(0, phase_t)[1]-0.02, m0_home_pos, m1_home_pos)[0]
m1_pos = from_pos_get_joint(RightStanceTrajectory(0, phase_t)[0], RightStanceTrajectory(0, phase_t)[1]-0.02, m0_home_pos, m1_home_pos)[1]
m00_pos= from_pos_get_joint(LeftSwingTrajectory(0, phase_t)[0], LeftSwingTrajectory(0, phase_t)[1]-0.01, m00_home_pos, m11_home_pos)[0]
m11_pos= from_pos_get_joint(LeftSwingTrajectory(0, phase_t)[0], LeftSwingTrajectory(0, phase_t)[1]-0.01, m00_home_pos, m11_home_pos)[1]

m0.controller.set_pos_setpoint(m0_pos,0,0)
m1.controller.set_pos_setpoint(m1_pos,0,0)
m00.controller.set_pos_setpoint(m00_pos,0,0)
m11.controller.set_pos_setpoint(m11_pos,0,0)

In [ ]:
# Run test-------------------------------------------------------------
k = 1
sr = 0.1
phase_t = 1 
t0 = time.time()

ControllerCurrentControl(odrv1)
ControllerCurrentControl(odrv2)

#(test_cycle, k_val, swing_t, stance_t, stiff_ratio, kill_rear_motor)
run_aniso_compliance_step_test(7,150,phase_t,phase_t,sr,k) 

print("time spent:{}sec".format(time.time() - t0))

In [ ]:
IdleMode(odrv1)
IdleMode(odrv2)

In [ ]:
print("odrive1")
CheckMotorError(odrv1)
print("odrive2")
CheckMotorError(odrv2)

MotorClearError(odrv1)
MotorClearError(odrv2)
print("Error Clear!")

In [ ]:
ControllerPositionControl(odrv1)
ControllerPositionControl(odrv2)

In [ ]:
ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)